In [1]:
#Just the basics to code sql
import psycopg2   as g2        #psyco must run with sqlalchemy
import sqlalchemy as sal       #needed for solid connection to postgres databases
import pandas     as pd        #needed for prettier sql result outputs
print("psycopg2:", g2.__version__)
print("sqlalchemy:", sal.__version__)
print("pandas:", pd.__version__)

psycopg2: 2.9.10 (dt dec pq3 ext lo64)
sqlalchemy: 2.0.38
pandas: 2.2.3


In [2]:
#Connect to psql w/ sal and g2 Note: Grok 3 says  ,text  is version related
from sqlalchemy import create_engine, text
user = "postgres"
password = "password"
host = "localhost"
port = "5433"
name = "rt"
pg = "postgresql"
#Create a string for the engine
c = f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{name}"
#Create engine
eg = create_engine(c)
#Test Connection
try:
    with eg.connect() as cn:
        result = cn.execute (text("SELECT version();"))
        print("BINGO!", result.fetchone()[0])
except Exception as e:
   print("Get to the Chawpaw!:", e)

BINGO! PostgreSQL 17.4 on x86_64-windows, compiled by msvc-19.42.34436, 64-bit


In [3]:
query = "SELECT * FROM match LIMIT 1;"
df = pd.read_sql(text(query),eg)
df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,sja,vch,vcd,vca,gbh,gbd,gba,bsh,bsd,bsa
0,1.0,1.0,1.0,2008/2009,1.0,2008-08-17 00:00:00,492473.0,9987.0,9993.0,1.0,...,4.0,1.65,3.4,4.5,1.78,3.25,4.0,1.73,3.4,4.2


In [4]:
######### Intro Section

In [5]:
# How many matches have been played in each of the 11 Leagues using instructors code
query = """
SELECT l.name AS league, COUNT(m.country_id) AS total_matches
FROM league AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
GROUP BY l.name;"""
df = pd.read_sql(text(query),eg)
df

,league,total_matches
0,Switzerland Super League,1422
1,Poland Ekstraklasa,1920
2,Netherlands Eredivisie,2448
3,Scotland Premier League,1824
4,France Ligue 1,3040
5,Spain LIGA BBVA,3040
6,Germany 1. Bundesliga,2448
7,Italy Serie A,3017
8,Portugal Liga ZON Sagres,2052
9,England Premier League,3040


In [6]:
query = """
SELECT *
FROM match
LIMIT 1
;"""
df = pd.read_sql(text(query),eg)
df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,sja,vch,vcd,vca,gbh,gbd,gba,bsh,bsd,bsa
0,1.0,1.0,1.0,2008/2009,1.0,2008-08-17 00:00:00,492473.0,9987.0,9993.0,1.0,...,4.0,1.65,3.4,4.5,1.78,3.25,4.0,1.73,3.4,4.2


In [7]:
query = """
SELECT *
FROM league
LIMIT 1
;"""
df = pd.read_sql(text(query),eg)
df

,id,country_id,name
0,1,1,Belgium Jupiler League


In [8]:
# How many matches have been played in each of the 11 Leagues using my code
# country_id is in both tables
# I need league name from league table

query = """
SELECT m.league_id, l.name, COUNT(league_id)
FROM match AS m
LEFT JOIN league AS l
ON m.country_id = l.country_id
GROUP BY m.league_id, l.name
ORDER BY COUNT(league_id) DESC
;"""
df = pd.read_sql(text(query),eg)
df

,league_id,name,count
0,21518.0,Spain LIGA BBVA,3040
1,4769.0,France Ligue 1,3040
2,1729.0,England Premier League,3040
3,10257.0,Italy Serie A,3017
4,13274.0,Netherlands Eredivisie,2448
5,7809.0,Germany 1. Bundesliga,2448
6,17642.0,Portugal Liga ZON Sagres,2052
7,15722.0,Poland Ekstraklasa,1920
8,19694.0,Scotland Premier League,1824
9,1.0,Belgium Jupiler League,1728


In [9]:
# Intro exercise CASE statment Home Team Wins in the 2013/2014 season

In [10]:
query = """
SELECT id, home_team_goal, away_team_goal,
    CASE WHEN home_team_goal > away_team_goal THEN 'Home Team Wins'
        WHEN  home_team_goal < away_team_goal THEN 'Away Team Wins'
        ELSE  'Tie' END AS outcome 
FROM match
WHERE season = '2013/2014'
LIMIT 10
;"""
df = pd.read_sql(text(query),eg)
df

,id,home_team_goal,away_team_goal,outcome
0,1237.0,2.0,0.0,Home Team Wins
1,1238.0,0.0,1.0,Away Team Wins
2,1239.0,1.0,0.0,Home Team Wins
3,1240.0,0.0,0.0,Tie
4,1241.0,2.0,1.0,Home Team Wins
5,1242.0,2.0,0.0,Home Team Wins
6,1243.0,2.0,4.0,Away Team Wins
7,1244.0,0.0,2.0,Away Team Wins
8,1245.0,4.0,2.0,Home Team Wins
9,1246.0,1.0,1.0,Tie


In [11]:
# The question here is to identify how many times FC Schalke 04 and FC Bayern Munich played each other.
# They was to answer this with a CASE statement. 
# The labels they provide are horrible

In [12]:
query = """
SELECT *
FROM match
LIMIT 1
;"""
df = pd.read_sql(text(query),eg)
df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,sja,vch,vcd,vca,gbh,gbd,gba,bsh,bsd,bsa
0,1283.0,1.0,1.0,2014/2015,13.0,2014-10-29 00:00:00,1717915.0,8573.0,10001.0,4.0,...,None,1.95,3.5,3.8,None,None,None,None,None,None


In [13]:
query = """
SELECT *
FROM team
WHERE team_long_name = 'FC Bayern Munich' OR team_long_name = 'FC Schalke 04'
;"""
df = pd.read_sql(text(query),eg)
df

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,15617,9823,21.0,FC Bayern Munich,BMU
1,15621,10189,34.0,FC Schalke 04,S04


In [14]:
# THEN creates a column, field name, or when GROUP BY because of count a record in a table

In [15]:
# I need to break this down
# GROUP BY is always going to create a count because COUNT is an aggregating function statement
# After the THEN is the records
# The logic used to answer the question is by eliminating all 'Other' the two remaining answer the question

In [16]:
query = """
SELECT
    CASE WHEN home_team_api_id = 10189 THEN 'FC Schalke 04'
    WHEN home_team_api_id = 9823 THEN 'FC Bayern Munich'
        ELSE 'Other' END AS home_team,
    COUNT (id) AS total_matches
FROM match
GROUP BY home_team
;"""
df = pd.read_sql(text(query),eg)
df

,home_team,total_matches
0,Other,25707
1,FC Schalke 04,136
2,FC Bayern Munich,136


In [17]:
# Create a list of matches in the 2011/2012 season where Barcelona what the home team
# GROUP BY outcomes; win, lose, tie
#
# The table I'm pulling from is match
# I need to Barcelona's ID or name to search by so I will need to pull from team table
# In SELECT I have to the season '2011/2012'
# Then I need to CASE

In [18]:
query = """
SELECT *
FROM team
WHERE team_long_name LIKE '%Barcelona'
;"""
df = pd.read_sql(text(query),eg)
df

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,43042,8634,241.0,FC Barcelona,BAR


In [19]:
query = """
SELECT *
FROM match
LIMIT 1
;"""
df = pd.read_sql(text(query),eg)
df

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,...,sja,vch,vcd,vca,gbh,gbd,gba,bsh,bsd,bsa
0,1283.0,1.0,1.0,2014/2015,13.0,2014-10-29 00:00:00,1717915.0,8573.0,10001.0,4.0,...,None,1.95,3.5,3.8,None,None,None,None,None,None


In [20]:
query = """
SELECT season, home_team_api_id
FROM match
WHERE season = '2011/2012' AND home_team_api_id = 8634
LIMIT 1
;"""
df = pd.read_sql(text(query),eg)
df

,season,home_team_api_id
0,2011/2012,8634.0


In [21]:
query = """
SELECT 
    CASE WHEN home_team_goal > away_team_goal THEN 'Barcelona win'
         WHEN home_team_goal < away_team_goal THEN 'Barcelona lost'
         ELSE 'Tie' END AS Outcome
FROM match
WHERE season = '2011/2012' AND home_team_api_id = 8634
;"""
df = pd.read_sql(text(query),eg)
df

,outcome
0,Barcelona win
1,Barcelona win
2,Barcelona win
3,Barcelona win
4,Barcelona win
5,Barcelona win
6,Barcelona win
7,Barcelona win
8,Barcelona win
9,Barcelona lost


In [22]:
# Re-do the above CASE statement and use LEFT JOIN to bring in the opponents name into this list

In [23]:
query = """
SELECT
    m.date, t.team_long_name AS opponent, 
        CASE WHEN home_team_goal > away_team_goal THEN 'Win' 
         WHEN home_team_goal < away_team_goal THEN 'Loss'
         ELSE 'Tie' END AS Outcome
FROM match AS m
LEFT JOIN team AS t
    ON m.away_team_api_id = t.team_api_id
WHERE m.season = '2011/2012' AND m.home_team_api_id = 8634
;"""
df = pd.read_sql(text(query),eg)
df

,date,opponent,outcome
0,2012-01-15 00:00:00,Real Betis Balompié,Win
1,2011-08-29 00:00:00,Villarreal CF,Win
2,2012-05-02 00:00:00,Málaga CF,Win
3,2012-02-04 00:00:00,Real Sociedad,Win
4,2012-02-19 00:00:00,Valencia CF,Win
5,2012-03-03 00:00:00,Real Sporting de Gijón,Win
6,2012-03-20 00:00:00,Granada CF,Win
7,2012-03-31 00:00:00,Athletic Club de Bilbao,Win
8,2012-04-10 00:00:00,Getafe CF,Win
9,2012-04-21 00:00:00,Real Madrid CF,Loss


In [ ]:
############################   In CASE things get more complex    ##################

In [ ]:
# Review: We are testing logical conditions
# Below looks at the logic to determine if the home team wins of loses or ties

In [26]:
query = """
SELECT
    date,
    season,
    home_team_api_id,
    away_team_api_id,
    CASE WHEN home_team_goal > away_team_goal THEN 'Home Team Win'
         WHEN home_team_goal < away_team_goal THEN 'Home Team Lost'
         ELSE 'Tie' END AS Outcome
FROM match
LIMIT 2;"""
df = pd.read_sql(text(query),eg)
df

,date,season,home_team_api_id,away_team_api_id,outcome
0,2012-01-07 00:00:00,2011/2012,8581.0,8661.0,Tie
1,2012-01-07 00:00:00,2011/2012,8633.0,7878.0,Home Team Win


In [ ]:
# Below looks at the logic to determine if Chelsea 8455 wins either as the away team or home team
# WHERE statement is left out

In [30]:
query = """
SELECT
    date,
    home_team_api_id,
    away_team_api_id,
    CASE WHEN home_team_api_id = 8455 AND home_team_goal > away_team_goal THEN 'Chelsea Home Win'
         WHEN away_team_api_id = 8455 AND away_team_goal > home_team_goal THEN 'Chelsea Away Win'
         ELSE 'Chelsea Lost or Tied' END AS Outcome         
FROM match;"""
df = pd.read_sql(text(query),eg)
df

,date,home_team_api_id,away_team_api_id,outcome
0,2012-01-07 00:00:00,8581.0,8661.0,Chelsea Lost or Tied
1,2012-01-07 00:00:00,8633.0,7878.0,Chelsea Lost or Tied
2,2012-01-08 00:00:00,8370.0,8302.0,Chelsea Lost or Tied
3,2012-01-08 00:00:00,8558.0,8634.0,Chelsea Lost or Tied
4,2013-05-22 00:00:00,9777.0,10179.0,Chelsea Lost or Tied
...,...,...,...,...
25974,2013-04-27 00:00:00,9956.0,10191.0,Chelsea Lost or Tied
25975,2011-11-29 00:00:00,8634.0,8370.0,Chelsea Lost or Tied
25976,2011-12-18 00:00:00,8371.0,10205.0,Chelsea Lost or Tied
25977,2011-12-17 00:00:00,8661.0,8305.0,Chelsea Lost or Tied


In [ ]:
# A CASE statement is mechanically not a filter, its a logic test, for each record
# For any record not match the logic SQL assigns the ELSE as the result

In [31]:
query = """
SELECT
    date,
    home_team_api_id,
    away_team_api_id,
    CASE WHEN home_team_api_id = 8455 AND home_team_goal > away_team_goal THEN 'Chelsea Home Win'
         WHEN away_team_api_id = 8455 AND away_team_goal > home_team_goal THEN 'Chelsea Away Win'
         ELSE 'Chelsea Lost or Tied' END AS Outcome         
FROM match
WHERE home_team_api_id = 8455 OR away_team_api_id = 8455;"""
df = pd.read_sql(text(query),eg)
df

,date,home_team_api_id,away_team_api_id,outcome
0,2008-08-17 00:00:00,8455.0,8462.0,Chelsea Home Win
1,2008-10-29 00:00:00,8667.0,8455.0,Chelsea Away Win
2,2008-11-01 00:00:00,8455.0,8472.0,Chelsea Home Win
3,2008-11-09 00:00:00,8655.0,8455.0,Chelsea Away Win
4,2008-11-15 00:00:00,8659.0,8455.0,Chelsea Away Win
...,...,...,...,...
299,2015-09-12 00:00:00,8668.0,8455.0,Chelsea Lost or Tied
300,2015-09-19 00:00:00,8455.0,9825.0,Chelsea Home Win
301,2015-09-26 00:00:00,10261.0,8455.0,Chelsea Lost or Tied
302,2015-10-03 00:00:00,8455.0,8466.0,Chelsea Lost or Tied


In [ ]:
# The same result can be achieved by using the entire CASE statement in the WHERE clause
# Just change the ELSE to END IS NOT NULL

In [36]:
query = """
SELECT
    date,
    home_team_api_id,
    away_team_api_id,
    CASE WHEN home_team_api_id = 8455 AND home_team_goal > away_team_goal THEN 'Chelsea Home Win'
         WHEN away_team_api_id = 8455 AND away_team_goal > home_team_goal THEN 'Chelsea Away Win'
         ELSE 'Chelsea Lost or Tied' END AS Outcome         
FROM match
WHERE    CASE WHEN home_team_api_id = 8455 AND home_team_goal > away_team_goal THEN 'Chelsea Home Win'
         WHEN away_team_api_id = 8455 AND away_team_goal > home_team_goal THEN 'Chelsea Away Win'
         END IS NOT NULL;"""
df = pd.read_sql(text(query),eg)
df

,date,home_team_api_id,away_team_api_id,outcome
0,2008-08-17 00:00:00,8455.0,8462.0,Chelsea Home Win
1,2008-10-29 00:00:00,8667.0,8455.0,Chelsea Away Win
2,2008-11-01 00:00:00,8455.0,8472.0,Chelsea Home Win
3,2008-11-09 00:00:00,8655.0,8455.0,Chelsea Away Win
4,2008-11-15 00:00:00,8659.0,8455.0,Chelsea Away Win
...,...,...,...,...
171,2015-08-23 00:00:00,8659.0,8455.0,Chelsea Away Win
172,2016-04-02 00:00:00,10252.0,8455.0,Chelsea Away Win
173,2016-04-23 00:00:00,8678.0,8455.0,Chelsea Away Win
174,2015-09-19 00:00:00,8455.0,9825.0,Chelsea Home Win


In [ ]:
# I need some code that manages PostgreSQL's driver psycopg2 or Pandas stack traces that explode when I mess up code

In [40]:
query = """
SELECT
    date,
    home_team_api_id,
    away_team_api_id,
    CASE WHEN home_team_api_id = 8455 AND home_team_goal > away_team_goal THEN 'Chelsea Home Win'
         WHEN away_team_api_id = 8455 AND away_team_goal > home_team_goal THEN 'Chelsea Away Win'
         ELSE 'Chelsea Lost or Tied' END AS Outcome         
FROM match
WHERE    CASE WHEN home_team_api_id = 8455 AND home_team_goal > away_team_goal THEN 'Chelsea Home Win'
         WHEN away_team_api_id = 8455 AND away_team_goal > home_team_goal THEN 'Chelsea Away Win'
         END IS NOT NULL;"""
try:
    # Execute the query
    df = pd.read_sql(text(query), eg)
    print(df)  # Or whatever you want to do with the dataframe
except pd.io.sql.DatabaseError as e:
    # Catch SQL-specific errors (e.g., syntax errors, table not found)
    print(f"SQL Error: {str(e).splitlines()[0]}")  # Prints just the first line of the error
except Exception as e:
    # Catch any other unexpected errors
    print(f"Unexpected Error: {str(e)}")

                    date  home_team_api_id  away_team_api_id           outcome
0    2008-08-17 00:00:00            8455.0            8462.0  Chelsea Home Win
1    2008-10-29 00:00:00            8667.0            8455.0  Chelsea Away Win
2    2008-11-01 00:00:00            8455.0            8472.0  Chelsea Home Win
3    2008-11-09 00:00:00            8655.0            8455.0  Chelsea Away Win
4    2008-11-15 00:00:00            8659.0            8455.0  Chelsea Away Win
..                   ...               ...               ...               ...
171  2015-08-23 00:00:00            8659.0            8455.0  Chelsea Away Win
172  2016-04-02 00:00:00           10252.0            8455.0  Chelsea Away Win
173  2016-04-23 00:00:00            8678.0            8455.0  Chelsea Away Win
174  2015-09-19 00:00:00            8455.0            9825.0  Chelsea Home Win
175  2015-10-17 00:00:00            8455.0           10252.0  Chelsea Home Win

[176 rows x 4 columns]


In [ ]:
# On to the exercises

In [ ]:
# Generate a list of matches played only between FC Barcelona and Real Madrid CF

In [59]:
query = """
SELECT
    date,
    CASE WHEN home_team_api_id = 8634 THEN 'FC Barcelona'
         ELSE 'Real Madrid CF' END AS Home,
    CASE WHEN away_team_api_id = 8634 THEN 'FC Barcelona'
         ELSE 'Real Madrid CF' END AS Away
FROM match
WHERE home_team_api_id = 8634 OR away_team_api_id = 8634
    AND home_team_api_id = 8633 OR away_team_api_id = 8633;"""
try:
    # Execute the query
    df = pd.read_sql(text(query), eg)
    print(df)  # Or whatever you want to do with the dataframe
except pd.io.sql.DatabaseError as e:
    # Catch SQL-specific errors (e.g., syntax errors, table not found)
    print(f"SQL Error: {str(e).splitlines()[0]}")  # Prints just the first line of the error
except Exception as e:
    # Catch any other unexpected errors
    print(f"Unexpected Error: {str(e)}")

                    date            home            away
0    2012-01-15 00:00:00    FC Barcelona  Real Madrid CF
1    2012-01-14 00:00:00  Real Madrid CF  Real Madrid CF
2    2011-08-29 00:00:00    FC Barcelona  Real Madrid CF
3    2011-08-28 00:00:00  Real Madrid CF  Real Madrid CF
4    2012-05-02 00:00:00  Real Madrid CF  Real Madrid CF
..                   ...             ...             ...
299  2011-12-03 00:00:00    FC Barcelona  Real Madrid CF
300  2011-12-03 00:00:00  Real Madrid CF  Real Madrid CF
301  2011-12-10 00:00:00  Real Madrid CF    FC Barcelona
302  2011-12-17 00:00:00  Real Madrid CF  Real Madrid CF
303  2011-11-29 00:00:00    FC Barcelona  Real Madrid CF

[304 rows x 3 columns]
